In [1]:
import pandas as pd
import urllib3 as urllib
import json
import IPython.display



In [2]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [3]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [4]:
#Saving a copy of source data 
ltc_data_json = json.dumps(ltc_data)

f = open("Source_data/IL_" + reporting_date + "LTC_data_Source.json","w")
f.write(ltc_data_json)
f.close()

In [5]:
df = pd.DataFrame(ltc_data['FacilityValues'])
df['reporting_date'] = reporting_date
df['% deaths'] = df['deaths'] / df['confirmed_cases']
df['num of facilities'] = 1 

In [6]:
df.to_csv('Reporting_data/IL_' + reporting_date + '_LTC_data_.csv', index = False)

In [7]:
a = df.sum()
print ('Date: %s' % reporting_date)
print ('Cases: %d' % a.confirmed_cases)
print ('Deaths:  %d' % a.deaths)

Date: 2020-05-29
Cases: 17100
Deaths:  2743


In [8]:
df_county = df.groupby(by=['County']).sum()
df_county['% deaths'] = (df_county['deaths'] / df_county['confirmed_cases'])
df_county.sort_values('deaths', ascending=False)

,confirmed_cases,deaths,% deaths,num of facilities
County,,,,
Cook,9890,1583,0.160061,259
DuPage,1751,297,0.169617,58
Lake,1249,175,0.140112,35
Will,744,143,0.192204,27
Kane,744,108,0.145161,27
McHenry,322,58,0.180124,10
St. Clair,320,50,0.156250,12
Madison,189,47,0.248677,5
Kankakee,259,32,0.123552,11


In [9]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O', sort='-x'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2

alt.HConcatChart(...)

In [10]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2 

alt.HConcatChart(...)

In [11]:
df.sort_values(by=['deaths'], ascending=False).to_csv('IL_LTC_Covid.csv')

In [12]:
df.sort_values(by=['deaths'], ascending=False)

,County,FacilityName,confirmed_cases,deaths,status,reporting_date,% deaths,num of facilities
493,Will,Meadowbrook Manor of Bolingbrook,188,40,Open,2020-05-29,0.212766,1
256,Cook,Villa at Windsor Park,153,37,Open,2020-05-29,0.241830,1
178,Cook,Niles Nursing and Rehab Center,158,33,Open,2020-05-29,0.208861,1
268,Cook,Woodbridge Nursing Pavilion,218,33,Open,2020-05-29,0.151376,1
193,Cook,Peterson Park Health Care,202,31,Open,2020-05-29,0.153465,1
...,...,...,...,...,...,...,...,...
319,DuPage,The Radcliff Wood Dale,8,0,Open,2020-05-29,0.000000,1
321,DuPage,West Chicago Terrace,9,0,Open,2020-05-29,0.000000,1
326,DuPage,Winfield Woods Healthcare Center,2,0,Open,2020-05-29,0.000000,1
327,DuPage,Wynscape Health and Rehabilitation,2,0,Open,2020-05-29,0.000000,1
